<a href="https://colab.research.google.com/github/kenjihiranabe/data-science-exercise/blob/main/101PCA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# PCA 101

In [ ]:
import numpy as np
import pandas as pd

from sklearn.datasets import load_wine
from sklearn.decomposition import PCA

pd.set_option("display.precision", 3)

In [ ]:
wine = load_wine(as_frame=True)["data"]
wine.shape

In [ ]:
# 最初の5列だけを利用する
# 5つの特徴量に対する主成分分析
A = wine.iloc[:,:5]
A.shape

In [ ]:
# 共分散
Cov = A.cov(ddof=0)
Cov

In [ ]:
# 共分散をnumpyで手計算
X = A.values
EX = X.mean(axis=0, keepdims=True) # 特徴量の平均ベクトル
EX

In [ ]:
# Cov(X) = E(X'X) - E(X)'E(X)
Cov = (X.T@X)/len(X) - EX.T@EX
pd.DataFrame(Cov)

## 中心化

In [ ]:
# 各列（特徴量）について、各特徴量の平均を減算することでデータを中心化します
# 線形代数的には、バイアスをなくして、原点からの線形性を保つ操作だと思います
n = len(X)
Xc = (np.eye(n) - np.ones((n, n))/n) @ X

## 固有値分解によるPCA

In [ ]:
lamb, S =  np.linalg.eig(Xc.T@Xc)

In [ ]:
list(np.sqrt(lamb))

In [ ]:
pd.DataFrame(S)

## SVDによるPCA

In [ ]:
U, sig, VT = np.linalg.svd(Xc, full_matrices=True)

In [ ]:
list(sig)

In [ ]:
pd.DataFrame(VT.T)

## sklearn.decomposition.PCAによるPCA

In [ ]:
pca = PCA()
pca.fit(X)

In [ ]:
# なぜか、手計算とは、値が若干異なる
# 最適化計算手法に依存するから？？？
list(pca.explained_variance_)

In [ ]:
pd.DataFrame(pca.components_.T)



```
# これはコードとして書式設定されます
```

# 2次元正規分布

- PCAは共分散行列からの固有値分解、あるいは特異値分解を利用して求める
- 共分散について考察します

In [ ]:
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns
 
mu = [0, 0]
sigma = [[10, 80], [80, 100]]
 
# 2次元正規乱数を生成
values = np.random.multivariate_normal(mu, sigma, 1000)
 
# 散布図
#sns.jointplot(values[:,0], values[:,1])
plt.gca().set_aspect("equal")
plt.scatter(values[:,0], values[:,1])
plt.grid()
plt.show()

## 固有値分解

In [ ]:
# AS = SΛ
lambs, S = np.linalg.eig(sigma)
lambs, S

In [ ]:
# 固有ベクトル上の値を生成
t = np.linspace(-30, 30, 100)
x1, y1 = S[0,0] * t,  S[1,0] * t # PC2
x2, y2 = S[0,1] * t,  S[1,1] * t # PC1

In [ ]:
# 固有ベクトル上の値をプロット
plt.gca().set_aspect("equal")
plt.scatter(values[:,0], values[:,1])
plt.scatter(x1, y1, label = "PC2")
plt.scatter(x2, y2, label = "PC1")
plt.grid()
plt.legend()
plt.show()

## 回転行列

In [ ]:
# 固有ベクトルは正規化されている
np.linalg.norm(S, axis=1)

In [ ]:
# 回転角度
theta = np.arctan2(S[1][0], S[0][0]) * 180 / np.pi
theta

In [ ]:
# 観測値を回転させる
rotated = values @ S

In [ ]:
plt.gca().set_aspect("equal")
plt.scatter(rotated[:,0], rotated[:,1])
# plt.scatter(x1, y1, label = "PC2")
# plt.scatter(x2, y2, label = "PC1")
plt.grid()
# plt.legend()
plt.show()

In [ ]:
# 固有ベクトル上の値も回転させる
xx1, yy1 = S.T @ np.vstack([x1, y1])
xx2, yy2 = S.T @ np.vstack([x2, y2])

In [ ]:
plt.gca().set_aspect("equal")
plt.scatter(rotated[:,0], rotated[:,1])
plt.scatter(xx1, yy1, label = "PC2")
plt.scatter(xx2, yy2, label = "PC1")
plt.grid()
plt.legend()
plt.show()